## Importing libraries and Data

In [345]:
import re
import pandas as pd
from datetime import datetime
from geopy.geocoders import Nominatim

In [346]:
# Load CSV files into DataFrames
users_df = pd.read_csv('../datasets/zomato/users.csv', index_col=[0])
restaurants_df = pd.read_csv('../datasets/zomato/restaurant.csv', index_col=[0])
orders_df = pd.read_csv('../datasets/zomato/orders.csv', index_col=[0])
menu_df = pd.read_csv('../datasets/zomato/menu.csv', index_col=[0])
food_df = pd.read_csv('../datasets/zomato/food.csv', index_col=[0])

C:\Users\taha\AppData\Local\Temp\ipykernel_31396\304725492.py:5: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  menu_df = pd.read_csv('../datasets/zomato/menu.csv', index_col=[0])


## Data Exploration

It seems we have a problem in column 5 shown by the following warning: *"DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False."*

In [347]:
menu_df

,menu_id,r_id,f_id,cuisine,price
0,mn0,567335,fd0,"Beverages,Pizzas",40.0
1,mn0,567335,fd669322,"Beverages,Pizzas",40.0
2,mn328,158203,fd0,Beverages,65.0
3,mn328,158203,fd669322,Beverages,65.0
4,mn449,158203,fd0,Beverages,65.0
...,...,...,...,...,...
1179931,mn1048555,96525,fd1048555,"Ice Cream,Desserts",300.0
1179932,mn1048556,96525,fd1048556,"Ice Cream,Desserts",300.0
1179933,mn1048558,96525,fd1048558,"Ice Cream,Desserts",350.0
1179934,mn1048561,96525,fd1048561,"Ice Cream,Desserts",200.0


In [348]:
# count for each value
menu_df.iloc[:, 4].value_counts()

price
120.0     28052
150.0     26758
100.0     23982
180.0     23222
140.0     22134
          ...  
402           1
466           1
12.8          1
89.6          1
305.68        1
Name: count, Length: 6071, dtype: int64

In [349]:
# price values containing "TWO" and ignore null values
menu_df[menu_df.iloc[:, 4].str.contains("TWO", na=False)]

,menu_id,r_id,f_id,cuisine,price
876935,mn413746,70147,fd413746,"North Indian, Chinese, Gujarati, Thalis, Birya...",₹200 FOR TWO
876936,mn413747,37556,fd413746,"American, Fast Food",₹350 FOR TWO
876937,mn413748,391189,fd413746,"Pizzas, Pastas, Beverages, Fast Food, Italian,...",₹400 FOR TWO
876938,mn413749,270210,fd413746,Desserts,₹500 FOR TWO
876939,mn413750,51684,fd413746,Chinese,₹200 FOR TWO
...,...,...,...,...,...
878123,mn414934,200757,fd413746,"Indian, Snacks, South Indian",₹250 FOR TWO
878124,mn414935,128936,fd413746,"Bakery, Desserts",₹700 FOR TWO
878125,mn414936,229306,fd413746,"Gujarati, North Indian",₹400 FOR TWO
878126,mn414937,553277,fd413746,"Indian, Chinese, Fast Food, Seafood",₹800 FOR TWO


So we have different types of values in the price column for example:</br>
120.0  =>  **Float**</br>
40  =>  **Integer**</br>
₹200 FOR TWO  => **String**</br>

## Data Processing

Let's fix the different types of values and transform the column into a Float type

In [350]:
# Function to extract and adjust the price
def extract_price(value):
    # Handling missing value
    if pd.isnull(value) or value == '':
        return None
    str_value = str(value)
    match = re.search(r'\d+', str_value)
    if match:
        price = float(match.group())
        # Check if the string contains 'FOR TWO' and adjust accordingly
        if 'FOR TWO' in str_value.upper():
            price /= 2
        return price
    else:
        return None 

# Convert the column to string before applying the function
menu_df.iloc[:, 4] = menu_df.iloc[:, 4].astype(str)

# Apply the function to the 'price' column
menu_df['price'] = menu_df.iloc[:, 4].apply(extract_price)

Checking values again

In [351]:
menu_df['price'].value_counts()

price
120.0     31696
150.0     30915
100.0     27322
180.0     26542
140.0     25054
          ...  
4099.0        1
1357.0        1
1644.0        1
2077.0        1
2550.0        1
Name: count, Length: 1498, dtype: int64

In [352]:
# Extract numeric part using regular expression
menu_df['menu_id'] = menu_df['menu_id'].str.extract('(\d+)')

# Convert the extracted strings to integers
menu_df['menu_id'] = menu_df['menu_id'].astype('Int64')

In [353]:
menu_df['r_id'] = menu_df['r_id'].astype('Int64')

Checking the column type

In [354]:
menu_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1179936 entries, 0 to 1179935
Data columns (total 5 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   menu_id  1179936 non-null  Int64  
 1   r_id     1179936 non-null  Int64  
 2   f_id     1179936 non-null  object 
 3   cuisine  1179936 non-null  object 
 4   price    1179936 non-null  float64
dtypes: Int64(2), float64(1), object(2)
memory usage: 56.3+ MB


In [355]:
orders_df['r_id'] = orders_df['r_id'].astype('Int64')

## DataFrames Creation

#### Countries DataFrame

In [12]:
# Initialize geocoder
geolocator = Nominatim(user_agent="geoapiExercises")

# Dictionary to store city-country mapping to avoid repeated geocoding
city_country_map = {}

def get_country(city, city_country_map):
    if city in city_country_map:
        return city_country_map[city]
    try:
        location = geolocator.geocode(city, addressdetails=True)
        country = location.raw['address']['country']
        city_country_map[city] = country
        return country
    except:
        return city
        

In [13]:
# Transform Data for countries Table
# Extracting unique cities from restaurant data and getting their respective countries

# Get unique cities and map them to countries
unique_cities = restaurants_df['city'].unique()
countries = [get_country(city, city_country_map) for city in unique_cities]


In [25]:
# Create countries DataFrame
countries_df = pd.DataFrame({'name': countries})
countries_df['created_at'] = datetime.now()


In [34]:
countries_df

,name,created_at
0,India,2024-01-07 17:48:48.044184
1,India,2024-01-07 17:48:48.044184
2,India,2024-01-07 17:48:48.044184
3,India,2024-01-07 17:48:48.044184
4,India,2024-01-07 17:48:48.044184
...,...,...
816,India,2024-01-07 17:48:48.044184
817,India,2024-01-07 17:48:48.044184
818,India,2024-01-07 17:48:48.044184
819,India,2024-01-07 17:48:48.044184


In [28]:
countries_df["name"].value_counts()

name
India            629
ایران              2
नेपाल              1
Cameroun           1
Indonesia          1
France             1
Zambia             1
United States      1
Name: count, dtype: int64

Most of the orders are from **India** so it makes **more sense** to do our analysis on a **City-Level** rather than **Country-Level**

#### Cities DataFrame

In [285]:
# Create cities DataFrame from unique cities in the restaurant data
cities_df = pd.DataFrame(restaurants_df['city'].unique(), columns=['name'])
cities_df['created_at'] = datetime.now()

# Remove None values or duplicates if any
cities_new = cities_df.dropna().drop_duplicates().reset_index(drop=True)

# Add an 'id' column (starting from 1 and incrementing for each row)
cities_new['id'] = range(1, len(cities_df) + 1)

In [286]:
cities_new

,name,created_at,id
0,Abohar,2024-01-07 20:44:18.962509,1
1,Adilabad,2024-01-07 20:44:18.962509,2
2,Adityapur,2024-01-07 20:44:18.962509,3
3,Adoni,2024-01-07 20:44:18.962509,4
4,Agartala,2024-01-07 20:44:18.962509,5
...,...,...,...
816,Warangal,2024-01-07 20:44:18.962509,817
817,Wardha,2024-01-07 20:44:18.962509,818
818,Washim,2024-01-07 20:44:18.962509,819
819,Yamuna-nagar,2024-01-07 20:44:18.962509,820


#### Stores DataFrame

In [356]:
# Map each city in restaurants_df to its city_id from cities_df
city_to_city_id = dict(zip(cities_df['name'], cities_new['id']))
restaurants_df['city_id'] = restaurants_df['city'].map(city_to_city_id)

# Create stores_df
stores_new = restaurants_df[['id', 'name', 'city_id']].copy()
stores_new.rename(columns={'name': 'slug'}, inplace=True)
stores_new['created_at'] = datetime.now()  # Assuming creation date is not available


In [357]:
stores_new

,id,slug,city_id,created_at
0,567335,AB FOODS POINT,1,2024-01-07 21:18:39.233653
1,531342,Janta Sweet House,1,2024-01-07 21:18:39.233653
2,158203,theka coffee desi,1,2024-01-07 21:18:39.233653
3,187912,Singh Hut,1,2024-01-07 21:18:39.233653
4,543530,GRILL MASTERS,1,2024-01-07 21:18:39.233653
...,...,...,...,...
148536,553122,The Food Delight,821,2024-01-07 21:18:39.233653
148537,562647,MAITRI FOODS & BEVERAGES,821,2024-01-07 21:18:39.233653
148538,559435,Cafe Bella Ciao,821,2024-01-07 21:18:39.233653
148539,418989,GRILL ZILLA,821,2024-01-07 21:18:39.233653


Let's transform the **slug** column to be more aligned with best practices

In [ ]:
# Function to create a slug from a string
def create_unique_slug(text, _id=None):
    # Convert to string in case of non-string types like float (NaN)
    text = str(text)
    # Convert to lowercase
    text = text.lower()
    # Replace non-word characters with hyphens
    text = re.sub(r'\W+', '-', text)
    # Replace multiple hyphens with a single hyphen
    text = re.sub(r'-+', '-', text)
    # Strip hyphens from the start and end of the text
    text = text.strip('-')
    # Append the id if provided
    unique_slug = f"{text}-{_id}" if _id is not None else text
    return unique_slug

# Tranform the slug column
stores_new['slug'] = stores_new['slug'].apply(lambda x: create_unique_slug(x))

#### Products DataFrame

In [358]:
# Merge menu_df with food_df to get the full product information
products_df = menu_df.merge(food_df, left_on='f_id', right_on='f_id')

In [359]:
# Map restaurant ID (r_id) from menu_df to store_id in stores_df
# Assuming stores_df has been populated with an 'id' column
r_id_to_store_id = dict(zip(restaurants_df['id'], stores_new['id']))
products_df['store_id'] = products_df['r_id'].map(r_id_to_store_id).astype('Int64')

In [360]:
# Select and rename relevant columns
products_new = products_df[['menu_id','item', 'price', 'store_id']].copy()
products_new.rename(columns={'menu_id': 'id', 'item': 'slug'}, inplace=True)

In [361]:
products_new

,id,slug,price,store_id
0,0,Aloo Tikki Burger,40.0,567335
1,328,Aloo Tikki Burger,65.0,158203
2,449,Aloo Tikki Burger,65.0,158203
3,729,Aloo Tikki Burger,60.0,158204
4,755,Aloo Tikki Burger,60.0,158204
...,...,...,...,...
1179931,1048555,Sizzlers De- Verdure,300.0,96525
1179932,1048556,Veg Bbq Sizzler,300.0,96525
1179933,1048558,Veg Sizzling Italian,350.0,96525
1179934,1048561,Veg Overload Sandwich,200.0,96525


In [ ]:
# Tranform the slug column
products_new['slug'] = products_new.apply(lambda x: create_unique_slug(x['slug'], x['id']), axis=1)

#### Order DataFrame

In [362]:
orders_df

,order_date,sales_qty,sales_amount,currency,user_id,r_id
0,2017-10-10,100,41241,INR,49226,567335
1,2018-05-08,3,-1,INR,77359,531342
2,2018-04-06,1,875,INR,5321,158203
3,2018-04-11,1,583,INR,21343,187912
4,2018-06-18,6,7176,INR,75378,543530
...,...,...,...,...,...,...
150276,2019-04-18,1,394,INR\r,79856,<NA>
150277,2019-04-23,1,667,INR\r,65798,<NA>
150278,2019-04-23,1,625,INR\r,49176,<NA>
150279,2019-04-24,7,8625,INR\r,87924,<NA>


In [363]:
# Merge restaurants_df with orders_df to get the full product information
orders_restaurant_df = orders_df.merge(restaurants_df, left_on='r_id', right_on='id')

In [364]:
orders_df.shape

(150281, 6)

In [365]:
orders_restaurant_df['cuisine'].value_counts()

cuisine
North Indian,Chinese         6475
Indian                       6417
Chinese                      5059
North Indian                 4779
Indian,Chinese               4382
                             ... 
Vietnamese,Pan-Asian            1
Healthy Food,Pan-Asian          1
Home Food,Naga                  1
Korean,Desserts                 1
Barbecue,Italian-American       1
Name: count, Length: 2132, dtype: int64

In [366]:
# Map r_id to store_id
# orders_df['store_id'] = orders_df['r_id'].map(r_id_to_store_id)
orders_new = orders_restaurant_df[['cuisine', 'r_id', 'order_date']].copy()
orders_new.rename(columns={'cuisine': 'type', 'r_id': 'store_id', 'order_date': 'created_at'}, inplace=True)


In [367]:
orders_new

,type,store_id,created_at
0,"Beverages,Pizzas",567335,2017-10-10
1,"Sweets,Bakery",531342,2018-05-08
2,Beverages,158203,2018-04-06
3,"Fast Food,Indian",187912,2018-04-11
4,"Italian-American,Fast Food",543530,2018-06-18
...,...,...,...
148659,"Fast Food,Snacks",553122,2019-01-22
148660,Pizzas,562647,2019-01-23
148661,"Fast Food,Snacks",559435,2019-01-23
148662,Continental,418989,2019-01-24


#### Order_items DataFrame

In [338]:
order_menu_df = pd.merge(orders_df, menu_df, left_index=True, right_index=True)

In [339]:
order_menu_df

,order_date,sales_qty,sales_amount,currency,user_id,r_id_x,menu_id,r_id_y,f_id,cuisine,price
0,2017-10-10,100,41241,INR,49226,567335,0,567335,fd0,"Beverages,Pizzas",40.0
1,2018-05-08,3,-1,INR,77359,531342,0,567335,fd669322,"Beverages,Pizzas",40.0
2,2018-04-06,1,875,INR,5321,158203,328,158203,fd0,Beverages,65.0
3,2018-04-11,1,583,INR,21343,187912,328,158203,fd669322,Beverages,65.0
4,2018-06-18,6,7176,INR,75378,543530,449,158203,fd0,Beverages,65.0
...,...,...,...,...,...,...,...,...,...,...,...
150276,2019-04-18,1,394,INR\r,79856,<NA>,283329,457682,fd663748,"North Indian,Chinese",120.0
150277,2019-04-23,1,667,INR\r,65798,<NA>,337219,36778,fd1237,Indian,90.0
150278,2019-04-23,1,625,INR\r,49176,<NA>,337219,36778,fd663748,Indian,90.0
150279,2019-04-24,7,8625,INR\r,87924,<NA>,350798,65039,fd1237,"Indian,Chinese",55.0


In [340]:
# Assuming a placeholder mapping for product_id
order_items_df = order_menu_df[['r_id_x', 'sales_qty']].copy()
order_items_df.rename(columns={'r_id_x': 'product_id', 'sales_qty': 'quantity'}, inplace=True)

In [342]:
order_items_df.dropna(inplace=True)

In [343]:
order_items_df

,product_id,quantity
0,567335,100
1,531342,3
2,158203,1
3,187912,1
4,543530,6
...,...,...
148659,390478,1
148660,103649,1
148661,430451,7
148662,518548,1
